<a href="https://colab.research.google.com/github/Krupa049/Text-Generation/blob/main/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
pip install numpy

In [19]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [4]:
text = text[300000:800000]

In [5]:
characters = sorted(set(text))

In [6]:
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [7]:
SEQ_LENGTH = 40
STEP_SIZE = 3

In [8]:
sentences = []
next_characters = []
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_characters.append(text[i + SEQ_LENGTH])

In [27]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool_)

In [28]:
for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_characters[i]]] = 1

In [29]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

In [30]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

In [31]:
model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
651/651 [==============================] - 103s 155ms/step - loss: 2.7132
Epoch 2/4
651/651 [==============================] - 101s 155ms/step - loss: 2.3124
Epoch 3/4
651/651 [==============================] - 102s 156ms/step - loss: 2.1977
Epoch 4/4
651/651 [==============================] - 101s 155ms/step - loss: 2.1154


In [32]:
model.save('textgenerator.model')

In [33]:
model = tf.keras.models.load_model('textgenerator.model')

In [34]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [35]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1

        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character

    return generated

In [36]:
print('------------0.2-----------')
print(generate_text(300, 0.2))
print('------------0.4-----------')
print(generate_text(300, 0.4))
print('------------0.6-----------')
print(generate_text(300, 0.6))
print('------------0.8-----------')
print(generate_text(300, 0.8))
print('------------1.0-----------')
print(generate_text(300, 1.0))

------------0.2-----------
xford, to prevent the worst,
forthwith whis be the hath heart the mand and the hard for the then the hand and and and and and and and and in the hard and the forthen the parenter the hare the mand and the hers and and and and and the sond and and the hing the mand
and that the mand the fare to the shand and that the beather the ward and a
------------0.4-----------
 love till now? forswear it, sight!
for my the hing that and in the beath that shand and is the enesther and heres of that sing
theall the herenthy wath sing wather soffrerest theer
betare the beate and ast and in the here
that wather forrencent,
the start to the proung ast and my and that the and in thas sist of the sing
there mistung to
------------0.6-----------
ld more care to keep
than in possession sanger
tofride
tho gale and ard of ande,
and linging hather mussed
for beagen, and that noth hame thes fars toas thenes,
and and menttofers for shent wath llother,
ind which hand thath my and ond g